In [11]:
import numpy as np 
import pandas as pd 
import geopandas as gpd 
from shapely.wkt import loads

In [7]:
# network
all_nodes = []
all_links = []
for tile_no in [533935, 533936, 533945, 533946]:
    nodes_tile = pd.read_csv('network_inputs/adf_nodes_{}.csv'.format(tile_no))
    links_tile = pd.read_csv('network_inputs/adf_links_{}.csv'.format(tile_no))
    all_nodes.append(nodes_tile)
    all_links.append(links_tile)
nodes_df = pd.concat(all_nodes)
links_df = pd.concat(all_links)

In [10]:
nodes_df.head(2)

,node_mapid,nb_node_mapid,ndatcd,geometry,nid,tile_no
0,533935_000C0,533945_0001D,4,POINT (139.6248409039033 35.66990345670202),43,533935
1,533935_001EE,533945_00089,4,POINT (139.6278656472181 35.66990353763025),48,533935


In [9]:
links_df.head(2)

,link_mapid,link_start_mapid,link_end_mapid,length,speed_limit_kmph,lanes,rdclasscd,geometry,fft,capacity,start_nid,end_nid,tile_no
0,533935_0001E_533934_0063D,533935_0001E,533934_0063D,167,80.0,3.0,3.0,LINESTRING (139.6227818128647 35.5892458020101...,7.515,6000.0,1596,0,533935
1,533935_00002_533935_00008,533935_00002,533935_00008,35,40.0,1.0,7.0,LINESTRING (139.6222317991397 35.5907539512060...,3.150,2000.0,1576,1581,533935


In [35]:
nodes_df_sim = nodes_df.copy()
nodes_df_sim = gpd.GeoDataFrame(nodes_df_sim, crs='epsg:4326', geometry=nodes_df_sim['geometry'].map(loads))
nodes_df_sim['lon'] = nodes_df_sim['geometry'].apply(lambda x: x.x)
nodes_df_sim['lat'] = nodes_df_sim['geometry'].apply(lambda x: x.y)
nodes_df_sim['node_id_igraph'] = np.arange(nodes_df_sim.shape[0])
nodes_df_sim = nodes_df_sim[['node_mapid', 'node_id_igraph', 'lon', 'lat']]
nodes_id_dict = {getattr(row, 'node_mapid'): getattr(row, 'node_id_igraph') for row in nodes_df_sim.itertuples()}

links_df_sim = links_df.copy()
links_df_sim['start_igraph'] = links_df_sim['link_start_mapid'].map(nodes_id_dict)
links_df_sim['end_igraph'] = links_df_sim['link_end_mapid'].map(nodes_id_dict)
links_df_sim = links_df_sim.dropna(subset=['start_igraph', 'end_igraph'])
links_df_sim['start_igraph'] = links_df_sim['start_igraph'].astype(int)
links_df_sim['end_igraph'] = links_df_sim['end_igraph'].astype(int)
links_df_sim['type'] = links_df_sim['rdclasscd']
links_df_sim = links_df_sim[['start_igraph', 'end_igraph', 'length', 'fft', 'capacity', 'type', 'geometry']]

nodes_df_sim.to_csv('network_inputs/nodes.csv', index=False)
links_df_sim.to_csv('network_inputs/links.csv', index=False)

In [36]:
print(nodes_df.shape, links_df.shape)

(117077, 6) (205748, 13)


In [15]:
for chunk_num in [0, 1, 2]:
    chunk_od = pd.read_csv('demand_inputs/od_osmid_res_above_{}.csv'.format(chunk_num))
    od_sim = chunk_od.copy()
    od_sim['node_id_igraph_O'] = od_sim['O'].map(nodes_id_dict)
    od_sim['node_id_igraph_D'] = od_sim['D'].map(nodes_id_dict)  
    od_sim['hour'] = od_sim['trip_hour']
    od_sim[['agent_id', 'node_id_igraph_O', 'node_id_igraph_D', 'hour']].to_csv('demand_inputs/od_{}.csv'.format(chunk_num), index=False)


In [31]:
nodes_df_sim[nodes_df_sim['node_id_igraph']==107161]

,node_mapid,node_id_igraph,lon,lat
26375,533946_734E2,107161,139.831183,35.743151


In [34]:
links_df_sim = links_df.copy()
links_df_sim['start_igraph'] = links_df_sim['link_start_mapid'].map(nodes_id_dict)
links_df_sim['end_igraph'] = links_df_sim['link_end_mapid'].map(nodes_id_dict)
links_df_sim = links_df_sim.dropna(subset=['start_igraph', 'end_igraph'])
links_df_sim[links_df_sim['link_mapid']=='533946_7343B_533946_735B1']

,link_mapid,link_start_mapid,link_end_mapid,length,speed_limit_kmph,lanes,rdclasscd,geometry,fft,capacity,start_nid,end_nid,tile_no,start_igraph,end_igraph
58063,533946_7343B_533946_735B1,533946_7343B,533946_735B1,23,40.0,1.0,NaN,LINESTRING (139.8307454891833 35.7430924218511...,2.07,2000.0,97985,98271,533946,107037,107323.0
